# 值迭代算法
作者：stzhao
github: https://github.com/zhaoshitian

# 一、定义环境


In [ ]:
# 导入numpy库，用于数值计算和数组操作
import numpy as np 

# 导入自定义环境DrunkenWalkEnv
# 这是一个简单的网格世界环境,模拟醉汉行走问题
# 醉汉在网格中移动,目标是到达终点位置
from envs.simple_grid import DrunkenWalkEnv

: 

In [4]:
def all_seed(env, seed=1):
    """设置所有随机数种子,确保实验的可重复性
    
    Args:
        env: 强化学习环境
        seed: 随机数种子,默认为1
        
    功能:
        - 设置环境的随机数种子
        - 设置numpy的随机数种子 
        - 设置Python random模块的随机数种子
        - 设置Python哈希种子
    """
    import numpy as np 
    import random 
    import os 
    # 设置环境的随机数种子
    env.seed(seed)
    # 设置numpy的随机数种子
    np.random.seed(seed)
    # 设置Python random模块的随机数种子 
    random.seed(seed)
    # 设置Python哈希种子
    os.environ['PATHONHASHSEED'] = str(seed)

In [5]:
# 创建醉汉行走环境实例
# map_name='theAlley'表示使用小巷地图
# theAlley地图是一个简单的网格世界,醉汉需要在其中导航到达目标
env = DrunkenWalkEnv(map_name='theAlley')

# 设置随机数种子以确保实验可重复性
# seed=1是一个任意选择的种子值
# 这将确保每次运行代码时得到相同的随机数序列
all_seed(env, seed=1)

c:\Code\Code_Practice\Reinforcement_Learning\Value Iteration\.conda\Lib\site-packages\gym\utils\seeding.py:41: DeprecationWarning: WARN: Function `rng.rand(*size)` is marked as deprecated and will be removed in the future. Please use `Generator.random(size)` instead.
  deprecation(


# 二、价值迭代算法


In [6]:
def value_iteration(env, theta=0.005, discount_factor=0.9):
    """
    实现价值迭代算法

    参数:
    env: 强化学习环境
    theta: 收敛阈值，默认为0.005
    discount_factor: 折扣因子，默认为0.9

    返回:
    Q: 最优动作价值函数
    """
    # 初始化Q表格，大小为[状态数, 动作数]
    Q = np.zeros([env.nS, env.nA])
    count = 0  # 迭代次数计数器

    while True:
        delta = 0.0  # 用于记录Q值的最大变化
        Q_tmp = np.zeros([env.nS, env.nA])  # 临时Q表格，用于存储更新后的Q值

        # 遍历所有状态和动作
        for state in range(env.nS):
            for action in range(env.nA):
                expected_reward = 0.0  # 期望奖励
                expected_value = 0.0  # 期望价值

                # 遍历当前状态-动作对的所有可能结果
                for prob, next_state, reward, done in env.P[state][action]:
                    expected_reward += prob * reward  # 计算期望奖励
                    expected_value += prob * max(Q[next_state, :])  # 计算期望价值

                # 更新Q值
                Q_tmp[state, action] = discount_factor * expected_value + expected_reward

                # 更新delta，记录最大Q值变化
                delta = max(delta, abs(Q_tmp[state, action] - Q[state, action]))

        Q = Q_tmp  # 更新Q表格
        count += 1  # 迭代次数加1

        # 检查是否收敛或达到最大迭代次数
        if delta < theta or count > 100:
            break

    return Q

In [7]:
# 使用价值迭代算法计算最优动作价值函数Q
Q = value_iteration(env)

# 打印最终的Q表格
# Q表格的形状为[状态数, 动作数]
# 每一行代表一个状态
# 每一列代表在该状态下采取相应动作的价值
print(Q)

[[2.25015697e+22 2.53142659e+22 4.50031394e+22 2.53142659e+22]
 [2.81269621e+22 5.41444021e+22 1.01257064e+23 5.41444021e+22]
 [6.32856648e+22 1.21824905e+23 2.27828393e+23 1.21824905e+23]
 [1.42392746e+23 2.74106036e+23 5.12613885e+23 2.74106036e+23]
 [3.20383678e+23 5.76690620e+23 1.15338124e+24 5.76690620e+23]
 [7.20863276e+23 1.38766181e+24 2.59510779e+24 1.38766181e+24]
 [1.62194237e+24 3.12223906e+24 5.83899253e+24 3.12223906e+24]
 [3.64937033e+24 7.02503789e+24 1.31377332e+25 7.02503789e+24]
 [8.21108325e+24 1.47799498e+25 2.95598997e+25 1.47799498e+25]
 [1.84749373e+25 3.55642543e+25 6.65097743e+25 3.55642543e+25]
 [4.15686089e+25 8.00195722e+25 1.49646992e+26 8.00195722e+25]
 [9.35293701e+25 1.80044037e+26 3.36705732e+26 1.80044037e+26]
 [5.89235032e+26 7.36543790e+26 7.57587898e+26 7.36543790e+26]]


In [16]:
# 初始化策略矩阵,大小为[状态数, 动作数]
# 每一行代表一个状态,每一列代表该状态下采取相应动作的概率
policy = np.zeros([env.nS, env.nA])

# 遍历所有状态
for state in range(env.nS):
    # 找出当前状态下价值最大的动作
    best_action = np.argmax(Q[state, :])
    # 将最优动作的概率设为1,其他动作概率为0
    policy[state, best_action] = 1

# 将策略矩阵转换为一维数组
# 对每个状态,找出概率为1的动作的索引
# np.argwhere返回非零元素的索引,取第一个索引[0]并转为整数
policy = [int(np.argwhere(policy[i] == 1)[0]) for i in range(env.nS)]
# 打印最终的确定性策略,每个数字代表在对应状态下应该采取的动作
print(policy)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


C:\Windows\Temp\ipykernel_22924\2001149784.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  policy = [int(np.argwhere(policy[i] == 1)[0]) for i in range(env.nS)]


# 三、测试

In [ ]:
num_episode = 1000 

def test(env)

In [10]:
test(env, policy)

测试的成功率是： 0.638
